In [ ]:
import geopandas as gpd

mainLink='https://github.com/DACSS-CSSmeths/Spatial-Exploring/raw/refs/heads/main/maps/countriesCIA.gpkg'
mapsLink=mainLink+'maps/worldMaps_Py.gpkg'

gpd.list_layers(mainLink)

cia=gpd.read_file(mainLink, layer='cia')
cia.head()

In [ ]:
!pip install mapclassify
import mapclassify

In [ ]:
import numpy as np

np.random.seed(12345)

K=5
theVar=cia.obesityAdults_rate


# same interval width, easy interpretation
ei5 = mapclassify.EqualInterval(theVar, k=K)
# same interval width based on standard deviation, easy - but not as the previous one, poor when high skewness
msd = mapclassify.StdMean(theVar)
# interval width varies, counts per interval are close, not easy to grasp, repeated values complicate cuts
q5=mapclassify.Quantiles(theVar,k=K)
# based on similarity, good for multimodal data
mb5 = mapclassify.MaximumBreaks(theVar, k=K)
# based on similarity, optimizer
fj5 = mapclassify.FisherJenks(theVar, k=K)
# based on similarity, optimizer
jc5 = mapclassify.JenksCaspall(theVar, k=K)
# based on similarity, optimizer
mp5 = mapclassify.MaxP(theVar, k=K)

###### based on similarity, good for skewed data
ht = mapclassify.HeadTailBreaks(theVar)

In [ ]:
import pandas as pd

class5 = ei5,msd, q5,mb5,  ht, fj5, jc5, mp5
# Collect ADCM for each classifier
fits = np.array([ c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms['classifier'] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ['ADCM', 'Classifier']

In [ ]:
import matplotlib.pyplot as plt
adcms.sort_values('ADCM').plot.barh(x='Classifier')
plt.show();

In [ ]:
fj5.yb

In [ ]:
cia['obesity_code']=fj5.yb

cia=cia.assign(obesity_levels=cia.obesity_code)

In [ ]:
newLevels={0:'1.very low',1:'2.low', 2:'3.average',3:'4.high', 4:'5.very high'}

cia.replace({'obesity_levels':newLevels}, inplace=True)

cia.obesity_levels.value_counts()

In [ ]:
countries=gpd.read_file(mainLink, layer='border')
worldBorders=countries.dissolve()
base=worldBorders.plot(color='lightgrey',
                     figsize=(15, 10))
cia.plot(column='tobacco_levels',ax=base,
                     cmap='PiYG_r', # reversed
                     categorical=True,
                     linewidth=0.,
                     legend=True,
                     legend_kwds={"title": "Obesity Rate (%)\n(missing data in grey)"}
                   )
base.set_axis_off()

# locating the legend
leg = base.get_legend()
leg.set_bbox_to_anchor((0.2, 0.6)) 

plt.show();

In [ ]:
topcountries=cia[(cia.tobacco_code==0) & (cia.alcohol_code==0) & (cia.obesity_code==0)]
topcountries

In [ ]:
cia.info()
###UNABLE TO SAVE OVER LAYERS AS SEEN IN LINE 57 OF TUTORIAL
###GIVES ERRROR UNABLE TO ACCESS DATABASE